In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd "/content/drive/MyDrive/Directed_studies"

/content/drive/MyDrive/Directed_studies


In [ ]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 kB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 58.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 51.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 56.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 86.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.5/409.5 kB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.2/76.2 k

In [ ]:
!pip install langchain_huggingface

In [ ]:
!pip install pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 9.0 MB/s eta 0:00:00


In [ ]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118


Looking in indexes: https://download.pytorch.org/whl/cu118


In [ ]:
import os
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import ConversationalRetrievalChain
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain_community.llms import CTransformers
import chainlit as cl
from langchain.memory import ConversationBufferMemory
from prompts_chat_pdf import chat_prompt, CONDENSE_QUESTION_PROMPT
os.environ["TOKENIZERS_PARALLELISM"] = "false"

class PDFChatBot:

    def __init__(self):
        self.data_path = os.path.join('data')
        print(self.data_path)
        self.db_faiss_path = os.path.join('vectordb', 'db_faiss')

    def create_vector_db(self):

        '''function to create vector db provided the pdf files'''
        if os.path.exists(self.db_faiss_path):
            # Load existing FAISS database
            embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2',
                                               model_kwargs={'device': 'cpu'})
            db = FAISS.load_local(self.db_faiss_path, embeddings, allow_dangerous_deserialization=True)
            print("Loaded existing FAISS database.")
        else:
            # Create a new FAISS database
            loader = DirectoryLoader(self.data_path, glob='*.pdf', loader_cls=PyPDFLoader)
            documents = loader.load()
            text_splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=50)
            texts = text_splitter.split_documents(documents)

            embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2',
                                               model_kwargs={'device': 'cpu'})
            db = FAISS.from_documents(texts, embeddings)
            db.save_local(self.db_faiss_path)
            print("Created and saved new FAISS database.")
        return db

    def load_llm(self):
        # Load the locally downloaded model here
        llm = CTransformers(
            model="llama-2-7b-chat.ggmlv3.q8_0.bin",
            model_type="llama",
            max_new_tokens=1500,  # Lower this value
            temperature=0.7
        )
        return llm

    def conversational_chain(self, question):

        embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2",
                                           model_kwargs={'device': 'cpu'})
        db = FAISS.load_local(self.db_faiss_path, embeddings, allow_dangerous_deserialization=True)

        # return conversational_chain
        retriever = db.as_retriever(search_kwargs={"k": 2})
        retrieved_docs = retriever.get_relevant_documents(question)
        retrieved_context = "\n".join([doc.page_content for doc in retrieved_docs])
        print(retrieved_context)
        llm = self.load_llm()
        response = llm.invoke(f"Use the context below to answer the question.\n\nContext:\n{retrieved_context}\n\nQuestion: "+question)
        print(response)





In [ ]:
def intialize_chain():
    bot = PDFChatBot()
    bot.create_vector_db()
    return bot

chat_history = []

bot = intialize_chain()
conversational_chain = bot.conversational_chain("What is cloud computing?")

data
Loaded existing FAISS database.
Cloud Computing
expertise in computing makes these services robust and their volume of business makes it 
cheaper for consumers.
REVIEW QUESTIONS 
What is meant by ‘cloud’ in cloud computing?
A cloud is combination of servers, storages, network devices (like switches etc.) networked together 
from where computing facilities are delivered to users. Cloud facility is developed by activating this


Answer: Cloud computing refers to the delivery of computing resources and services over the internet, allowing users to access and use computer systems, servers, storage devices, and applications remotely. It provides on-demand access to a range of computing resources, including servers, storage, databases, software, and applications, which can be easily scaled up or down as needed. The cloud computing model is based on the concept of a "cloud," which refers to a network of interconnected servers that work together to provide computing resources and services

In [ ]:
conversational_chain = bot.conversational_chain("What is pay as you use model?")


fixed rental model, a fixed amount is charged depending on subscriber’s need for some fixed 
duration (generally counted in months). In actual use-basis approach, billing is done as per 
actual consumption of the service by a subscriber over a period.
Utility computing model is the implementation of utility model of service delivery in computing.
public utilities like electricity, water, or telephone. The model follows pay-per-use mode of 
payment and on-demand service facility.
In utility model , computing vendors own, build and manage the computing resources and 
infrastructure. They deliver any kind of computing facilities that customers can access as and
by cloud service providers?
On-demand VM subscription model is the general subscription model of cloud computing where 
payment is made purely based on the usage basis. When consumer does not use the service he/she 
need not to pay anything. There is no prior agreement of service use.
In reserved VM subscription model, consumers co

In [ ]:
conversational_chain = bot.conversational_chain("What are the advantages of using cloud computing??")


set of points briefing the benefits of cloud computing can be as follows:
 1.  Achieve economies of scale: We can increase the volume output or pro-
ductivity with fewer systems and thereby reduce the cost per unit of 
a project or product.
 2. Reduce spending on technology infrastructure : It is easy to access data 
and information with minimal upfront spending in a pay-as-you-go
This long list of benefits discussed above are represented in Figure 3.2. They give some idea 
about the usefulness of cloud computing and why so many people are excited about it.
Automatic Software Updates
No License Procurement
Quick Deploymen
t
Locational Independence
Continuous Availability
Reliability
Quality of Service
Less System Management Liability
Use-basis Payment Facility
Therefore, cloud computing is needed in getting the services of comput -
ing resources. Thus, one can say as a one-line answer to the need for cloud 
computing that it eliminates a large computing investment without compro-
misin

In [ ]:
bot.conversational_chain("What are the main types of cloud computing services?")

main categories of cloud services? 
 a)  Software-as-a-Service
 b)  Database-as-a-Service 
 c)  Platform-as-a-Service
 d) Infrastructure-as-a-Service
 8. Which cloud computing service model 
delivers computer, storage, and network to 
consumers?
 a)  Infrastructure-as-a-Service 
 b)  Platform-as-a-Service
 c)  Software-as-a-Service
 d) All of these
 9.  Retaining with traditional computing
cloud computing. Then, it deals with cloud services types, cloud deploy -
ment models, technologies supporting and driving the cloud, software 
process models and programming models for cloud, and development of 
software application that runs the cloud. It also gives an overview of ser -
vices available from major cloud providers, highlights currently available
